# Kriging in python

In [1]:
import numpy as np
from scipy.spatial.distance import pdist, squareform
import geostat as geo
import matplotlib.pyplot as plt

A data set

In [2]:
# Input data: the measurements
x1 = np.array( [61,63,64,68,71,73,75] )        # Locations of the points (x1=x, x2=y)
x2 = np.array( [139,140,129,128,140,141,128] )  
v = np.array( [477,96,227,646,606,791,783] )   # Value of the measurements 

# The data in matrix form = one line per data point
x = np.transpose( (x1,x2) )

# The locations where we want to estimate 
x10 = np.array([68, 66, 69]); 
x20 = np.array([135, 132, 134]);
xu = np.transpose( (x10,x20) )

A simple data set

In [ ]:
# Input data: the measurements
x1 = np.array( [61,63,64] )    # Loations of the points (x1=x, x2=y)
x2 = np.array( [139,140,129] )  
v = np.array( [477,96,227] )   # Value of the measurements 

# The data in matrix form = one line per data point
x = np.transpose( (x1,x2) )

# The locations where we want to estimate 
x10 = np.array([62, 63]); 
x20 = np.array([135, 133]);
xu = np.transpose( (x10,x20) )

In [ ]:
# Plot the data for visual check

plt.plot(x1,x2,'+');   # Position des points
plt.plot(x10,x20,'or') # Point où l'on veut estimer
plt.show()

The ordinary kriging equation

In [ ]:
# Defines the type of covariance function and parameters
covar = geo.covariance(rang=10, sill=20,typ="spherical")

# Defines the size of the problem and agglomerate the data
n_data = len(v)
n_unknown = len(x10)
n_total = n_unknown + n_data
xall = np.concatenate((xu,x))

# Computes the covariance for all pairs of nodes
d = squareform( pdist(xall) )
c = covar(d)

# Assemble the Kriging matrix 
a = np.ones( (n_data+1,n_data+1) )
a[0:n_data,0:n_data] = c[n_unknown:n_total,n_unknown:n_total]  
a[n_data,n_data] = 0

# Assemble the right hand side 
b = np.ones( (n_data+1,n_unknown) )
b[0:n_data,0:n_unknown] = c[n_unknown:n_total,0:n_unknown]

# Solve the Kriging system
l = np.dot(np.linalg.inv(a), b)

# Get the Kriging weight for each data point 
w = l[0:n_data,0:n_unknown]

# Prepare the calculation of Kriging values 
v.shape=(n_data,1)

# Computes the Estimated values by multiplying the kriging weights with values
vo = np.dot( np.transpose(w), v)

# Get the Lagrange parameter for each unknown point
mu = l[-1,:] 
mu.shape=(n_unknown,1)

# Computes the kriging variance for each unknown point
so = np.diag( np.dot( np.transpose(w), b[0:n_data,:]) )
so.shape = (n_unknown,1)
so = so + covar._sill - mu

print('Estimation:',vo)

print('Variance:',so)
